# Random Walks

In [ ]:
import numpy as np                # Import NumPy package
import matplotlib.pyplot as plt   # Import Matplotlib package to plot graphs
plt.rcParams['figure.dpi'] = 100  # Set the figure dpi to 100 globally

from numba import njit # Uncomment this line if you want to use Numba to speed up your code.

In [ ]:
@njit  # Uncomment this line if you want to use Numba to speed up your code.
def walk(n_steps=10_000, bc=True, L=30):
    '''
    A function produces a single random walk with a specific number of steps.

    Parameters:
    -----------
    • n_steps : The number of steps in the random walk.
    • bc      : A boolean variable that decides if periodic boundary conditions in a box are enforced or not.
    • L       : The box-size is given by 2L if periodic boundary conditions are enforced.

    Returns:
    --------
    The function returns one value:
    • r: An (n_steps x 2) array containing each individual step of the random walk. 
         Each step has 2 coordinates, i.e. (x,y).
    '''
    
    r = np.zeros(shape=(n_steps, 2), dtype=np.int32)  # A 2D array of zeros to store the 
                                                      # (x,y) coordinates of each of the N steps
    x,y = 0,1                                         # Shorthand for the indices x and y: 
                                                      # the first row is x, and the second is y
    
    
    for i in range(1,n_steps):                        # For each step,
        
        # First find out if the walker is moving forward or backwards delta > 0 => forwards
        delta = -1 if (np.random.uniform(0,1)< 0.5) else +1

        # Next, find out if the walker is moving left-right or top-bottom. Choose with equal probability
        if(np.random.uniform(0,1)<0.5):
            r[i][x] = r[i-1][x] + delta  # If left-right (i.e. along x), old x position is incremented by delta
            r[i][y] = r[i-1][y]          # But old y position stays the same
        else: 
            r[i][x] = r[i-1][x]          # Similarly if it's top-bottom (i.e. along y), x position stays the same
            r[i][y] = r[i-1][y] + delta  # y position is incremented by delta
        
        
        # Check to see if we need to implement periodic boundary conditions
        if(bc):                          # If yes, the
            if(r[i][x] > L):             # If the x position is greater than the box-size, 
                r[i][x] = r[i][x] - 2*L  # then change x -> x-2L
            elif(r[i][x] < -L):          # If the x position is less than the box-size, 
                r[i][x] = r[i][x] + 2*L  # then change x -> x + 2L

            if(r[i][y] > L):             # Similarly for the y position, make sure it's between -L and L
                r[i][y] = r[i][y] - 2*L
            elif(r[i][y] < -L):
                r[i][y] = r[i][y] + 2*L
        
    return r                              # Finally, return the history of this random walk

In [ ]:
# This function has been written for you, you don't need to edit it. 
def simulate(n_walks = 50, n_steps = 10_000, bc=False, L=30):
    '''
    A function to simulate multiple random walks of some number of steps, with or without boundary conditions.
    This function has been written for you, you don't need to edit it unless you want to.

    Parameters:
    -----------
    • n_walks : The number of random walks to simulate.
    • n_steps : The number of steps in each random walk.
    • bc      : A boolean variable that decides if periodic boundary conditions in a box are enforced or not.
    • L       : The box-size is given by 2L if periodic boundary conditions are enforced.

    Returns:
    --------
    The function returns one value:
    • all_walks: An (n_walks x n_steps x 2) array containing each individual step of each random walk. 
                 The first dimension is the walk-number, the second the step-number, and the third the 
                 third the coordinate (x or y).
    '''
    
    all_walks = np.zeros((n_walks, n_steps, 2))   # Array to store data of each walk

    for w in range(n_walks):                      # For each walk,

        cmap = plt.colormaps['viridis']           # Each colour will be chosen from the `viridis` cmap, with
        d_color = 1/n_walks                       # "spacing" between colours of 1/n_walks, so the full map is used

        rw = walk(n_steps=n_steps, bc=bc, L=L)    # Simulate a random walk ith required parameters
        all_walks[w] = rw                         # Store the simulated walk in `all_walks`
        
        # Plot the random walk
        plt.plot(rw[:,0], rw[:,1], marker='o', ms=0.01, color=cmap(w*d_color), alpha=1)
    
    plt.ylabel(r"$y$")
    plt.xlabel(r"$x$")

    # Plot the origin of the walks. The zorder variable decides which plot is plotted after which one,
    # and makes sure that the scatter point is visible and not plotted _underneath_ the other plots.
    plt.scatter(0, 0, color='firebrick', s=10, zorder=n_walks+1)        
    plt.gca().set_aspect('equal') # Set the aspect ratio of the axis to equal, so a square looks like a square
    
    return all_walks # Return the final array of all the walks

In [ ]:
%%time
# This cell stores all the walk data for 50 walks of 10_000 steps each in `all_walks`. It should take about
# 10 seconds to run, if no errors have been made.
all_walks = simulate(n_walks=50)

In [ ]:
def plot_evolution(all_walks, plot_L=200):
    '''
    A function to plot the evolution of generated multiple random walks.
    This function has been written for you, you don't need to edit it unless you want to.

    Parameters:
    -----------
    • all_walks : Position data of a group of random walks, as a 3D array of dimension (n_walks x n_steps x 2).
    • plot_L    : x and y limits of the resulting plot are taken to be -plot_L and plot_L respectively.

    Returns:
    --------
    NoneType (A plot).

    '''
    
    n_walks = len(all_walks)    # Number of walks
    n_steps = len(all_walks[0]) # Number of steps per walk
    
    ncols=5                     # Number of intervals. Plotted as columns in a figure below
    
    del_steps = n_steps//ncols  # Steps per interval. The double-slash indicates "integer-division" eg 5//3=1.

    # Define a figure and a set of axes of 1 row and 5 columns (default) and some figure size
    fig, axes = plt.subplots(nrows=1, ncols=ncols, figsize=(4*ncols,4))

    cmap = plt.colormaps['viridis'] # Set the colour map
    d_color = 1/n_walks             # and the interval between colours, as before, so that the full map is used

    axes[0].set_ylabel(r"$y$")
    
    for i in range(ncols):          # For each interval,

        axes[i].scatter(0, 0, color='firebrick', s=10, zorder=n_walks+1)  # Plot the start-point of the walk

        # "Clip" the walk so that the first `(i+1)*del_steps` steps is chosen for each walk.
        # This will look confusing for those of you not familiar with NumPy, but you will get used to it by the
        # end of the course! You don't _need_ to write code like this, but it makes things faster.
        clipped_walks = all_walks[:,:(i+1)*del_steps,:]                   
        
        # For each "clipped walk"
        for j in range(n_walks):
            # Plot the walk in the appropriate column of the figure (i.e. in axes[i]) with a different colour
            axes[i].scatter(clipped_walks[j,:,0], clipped_walks[j,:,1],  s=0.1, color=cmap(j*d_color), alpha=0.1, zorder=i)
        
        axes[i].set_xlim(-plot_L,plot_L) # Set the axes limits based on the `plot_L` parameter
        axes[i].set_ylim(-plot_L,plot_L) # The result is always square
        axes[i].set_aspect('equal')      # Set the axes aspect ratio to be square
        axes[i].set_xlabel(r"$x$")       # Set the x-label for each plot
        
        
    plt.tight_layout()

In [ ]:
%%time
# This cell plots the evolution of the walks stored in the `all_walks` variable created above.
plot_evolution(all_walks)

<div class="alert alert-block alert-warning">
<b>Comment:</b> As you can see, a "bundle" of walks is produced. Each random walk on its own is not spherically symmetric -- in fact, each walker generally seems less likely to return towards the origin as time progresses -- but the bundle certainly is, which is a reflection of the fact that the mechanism that creates such a walk does not favour any particular direction. Additionally, while the walk initially seems to grow noticeably, the returns are diminishing, and it is rather the area of the bundle that seems to scale up with time rather than the radius. This is corroborated by the result in the next part of the question.
</div>

In [ ]:
def get_Rsq(walk):
    '''
    A function get how the R-squared values change over the evolution of a single random walk.

    Parameters:
    -----------
    • walk : Position data of a single of random walk, as a 2D array of dimension (n_steps x 2).

    Returns:
    --------
    The function returns a single value:
    • Rsq  : Array of same length as `walk` with R-squared values at every point between the 
             start and the end of the walk.

    '''  
    
    Rsq = np.zeros(len(walk))
    
    for i in range(len(Rsq)):
        Rsq[i] = np.sum((walk[i] - walk[0])**2)
        
    return Rsq

In [ ]:
# Finding R-squared values for all walks

all_Rsq = np.zeros((len(all_walks), len(all_walks[0])))  # Define array to store R-squared data for each walk

for w in range(len(all_walks)):                          # For each walk, get and store the R-squared data
    all_Rsq[w] = get_Rsq(walk=all_walks[w])       


In [ ]:
def get_trendline(data_x, data_y):
    '''
    Find the trendline between x and y data.

    Parameters:
    -----------
    • data_x : x-coordinate data.
    • data_y : y-coordinate data.

    Returns:
    --------
    The function returns a single value:
    • z : A 1d polynomial class of the form z = m*x + c. To use it, simply say z(x), where x is a numpy array
          of x coordinates.
    '''
    
    z = np.poly1d(np.polyfit(data_x, data_y, deg=1))
    return z

In [ ]:
# Plot the R-squared data (averaged over _walks_) as a function of the number of steps.
def plot_Rsq(all_Rsq, plot_skip=200, low_clip=1, plotloglog=False, color='cadetblue', ec='navy', s=20):
    '''
    Plot Rsq as a function of steps.

    Parameters:
    -----------
    • all_Rsq   : Array containing rsquared values for multiple walks. 
                  Rsquared values are assumed to be for every step.
    • plot_skip : (optional) Number of points to skip between successive plotted points (to avoid crowded graphs).
    • low_clip  : (optional) Clip the lower end of the graph (for periodic boundary conditions, for example).
    • plotloglog: (optional) Boolean to plot a log-log graph and trendline, to see how <R_d^2> varies in time. 
    • color     : (optional) Colour of the scatter plot. (Default is 'cadetblue'.)
    • ec        : (optional) Edge colour of the scatter plot. (Default is 'navy'.)
    • s         : (optional) Marker size of the scatter plot. (Default is 20.)

    Returns:
    --------
    NoneType (A plot).
    '''
    
    n_steps = len(all_Rsq[0]) # Number of steps in each random-walk

    steps = np.arange(low_clip,n_steps,1) # All the steps from low_clip to n_steps

    averaged_Rsqs = np.mean(all_Rsq, axis=0) # Create an averaged array by summing over the "zero-th" axis
        
    # Scatter-plot the R-squared data averaged over multiple walks
    # Since this is a lot of data, I am only going to _plot_ every (say) 200th point. This is done by indexing
    # the NumPy array using "[::200]". This means: from start to finish, in steps of 200.
    x_data = steps[::plot_skip]
    y_data = averaged_Rsqs[low_clip:][::plot_skip]

    if(plotloglog): # If we're plotting in a log-scale, use the log data instead
        x_data = np.log10(x_data)
        y_data = np.log10(y_data)

    plt.scatter(x_data, y_data, color=color, ec=ec, s=s) # Scatter plot the data

    # Compute the trendline parameters
    z = get_trendline(x_data, y_data)   # Use the `get_trendline` function to obtain the fit
    m,c = z.coefficients                # Get the coefficients of the fit

    # Plot parameters, depending on the type of plot requested
    if(plotloglog):
        plotlabel = r"$"+str(round(10**c,2))+" \cdot t^{"+str(round(m,2))+"}$"
        xlabel = "Log Steps"
        ylabel = r"Log $\langle R_d^2 \rangle$"
    else:
        if(c<0):          # To take care of the case when the intercept is zero, if the intercept is negative,
            sign = ""     # don't add a "sign" string.
        else:
            sign = "+"    # Otherwise, add a sign string that's a "+" in the label
            
        plotlabel = str(round(m,2))+" t "+sign+str(round(c,2))
        xlabel = "Steps"
        ylabel = r"$\langle R_d^2 \rangle$"
        
    plt.plot(x_data, z(x_data), color='firebrick', ls='--', label=plotlabel) # Plot trendline

    plt.legend()
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)

In [ ]:
plot_Rsq(all_Rsq)

<div class="alert alert-block alert-warning">
<b>Comment:</b> The average value of $\langle R_d^2 \rangle$ is plotted for every 200th walk. The slope of the linear graph tells us that the diffusion coefficient satisfies $4D\approx 1$, or $D=1/4$ in this case.
</div>

In [ ]:
# Plotting a log-log plot
plot_Rsq(all_Rsq, low_clip=1000,plotloglog=True, color='grey', ec='black')

<div class="alert alert-block alert-warning">
<b>Comment:</b> The average value of $\langle R_d^2 \rangle$ is plotted for every 200th walk on a log-log scale. To justify our claim that the functional relation is linear, we also plot a log-log plot above. Since the range of values spans multiple orders on both axes (1e1 to 1e4), the log-log plot can be trusted to give a close functional approximation to a power law, if such a relationship exists. The power of $t$ is very close to 1 (around 1.11) within the error expected for only 50 random walks.
</div>

In [ ]:
%%time
# This cell stores all the walk data for 50 walks of 10_000 steps each in `all_walks`. It should take about
# 10 seconds to run, if no errors have been made.
all_pbc_walks = simulate(n_walks=50, n_steps=10_000,bc=True, L=30)

<div class="alert alert-block alert-warning">
<b>Comment:</b> As expected, with periodic boundary conditions, the walks densely fill up the box, as ink would in a bounded container.
</div>

In [ ]:
plot_evolution(all_pbc_walks, plot_L=35)

<div class="alert alert-block alert-warning">
<b>Comment:</b> This is further evidenced in the time evolution, where we see the "density" slowly become uniform as the walks progress further in time.
</div>

In [ ]:
all_pbc_Rsq = np.zeros((len(all_pbc_walks), len(all_pbc_walks[0])))  # Define array to store R-squared data for each walk

for w in range(len(all_pbc_walks)):                    # For each walk, get and store the R-squared data
    all_pbc_Rsq[w] = get_Rsq(walk=all_pbc_walks[w])       

plot_Rsq(all_pbc_Rsq,low_clip=1, plot_skip=10, s=1)    # Plot <R_d^2> as a function of step (ignoring the first 1000 steps)

<div class="alert alert-block alert-warning">
<b>Comment:</b> It was found that the value of $\langle R_d^2 \rangle$ seemed to stabilise after an initial (linear) rise. To study the value to which it stabilised, a plot was made, and this value was found to be roughly close to $\sim 600$. This is particularly interesting, since in a uniformly filled boxed, the average value of the squared distance $r^2$ can be shown to be 
    $$\langle r^2 \rangle = \frac{1}{L^2} \int_{0}^{L} \mathrm{d} x \int_{0}^{L} \mathrm{d} y\,\, (x^2 + y^2) = \frac{2L^2}{3}.$$
    For $L=30$, $\langle r^2 \rangle = 600$. This is a sign that the random walks truly fill up the space uniformly, just as a drop of ink fills up a container uniformly.
</div>